In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans


In [ ]:
df = pd.read_csv('/kaggle/input/ecommerce-data/data.csv')
df.head(5)

In [ ]:
df.info()

In [ ]:
def count(df,feature):
    total=df.loc[:,feature].value_counts(dropna=False)
    percent=round(df.loc[:,feature].value_counts(dropna=False,normalize=True),2)*100
    return pd.concat([total,percent],axis=1,keys=['Total','Percent'])
    

In [ ]:
count(df,'Country').T

In [ ]:
plt.figure(figsize=(15,15))
sns.countplot(x="Country", data=df)
locs, labels = plt.xticks()
plt.setp(labels, rotation=90)
plt.show()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df=df.drop_duplicates().reset_index(drop=True)

In [ ]:
df['TotalPrice']=df['UnitPrice']*df['UnitPrice']

In [ ]:
df_country=df.groupby('Country').agg({'TotalPrice': lambda x: x.sum()})

In [ ]:
df_country.T

In [ ]:
import datetime as dt


In [ ]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [ ]:
df['InvoiceDay'] = df['InvoiceDate'].apply(lambda x: dt.datetime(x.year, x.month, x.day))

In [ ]:
pin_df=max(df['InvoiceDay'])+dt.timedelta(1)
pin_df

In [ ]:
rfm=df.groupby('CustomerID').agg({'InvoiceDay':lambda x:(pin_df-x.max()).days,'InvoiceNo':'count','TotalPrice':'sum'})

In [ ]:
rfm

In [ ]:
rfm.rename(columns= {
    'InvoiceDay': 'Recency',
    'InvoiceNo': 'Frequency',
    'TotalPrice': 'Monetary'
}, inplace=True)
rfm

In [ ]:
r_labels=range(4,0,-1)
r_groups=pd.qcut(rfm['Recency'],q=4,labels=r_labels)
f_groups=pd.qcut(rfm['Frequency'],q=4,labels=r_labels)
m_groups=pd.qcut(rfm['Monetary'],q=4,labels=r_labels)


In [ ]:
rfm['R']=r_groups.values
rfm['F']=f_groups.values
rfm['M']=m_groups.values


In [ ]:
rfm

In [ ]:
X = rfm[['R', 'F', 'M']]

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
plt.figure(figsize=(15,10))
distortions=[]
sil_scores=[]
for i in range(2,10):
    kmeans= KMeans(n_clusters=i, n_init=10, init= 'k-means++', algorithm='full', max_iter=300)
    kmeans.fit(X)
    distortions.append(kmeans.inertia_)
    label= kmeans.labels_
    sil_scores.append(silhouette_score(X, label))
plt.plot(np.arange(2,10,1), distortions, alpha=0.5)
plt.plot(np.arange(2,10,1), distortions,'o' ,alpha=0.5)
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=5, init='k-means++', max_iter=300)
kmeans.fit(X)

In [ ]:
kmeans.labels_

In [ ]:
rfm['kmeans_cluster'] = kmeans.labels_

In [ ]:
rfm[rfm['kmeans_cluster'] == 4].mean()

In [ ]:
rfm.mean()

In [ ]:
kmeans.inertia_